# Carga da Dimensão Ativo Financeiro

Este notebook realiza a carga da dimensão ativo financeiro (dim_ativo_financeiro) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [8]:
from spark_config import init_spark
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql import Window

## Start Spark Session

In [9]:
spark = init_spark("Carga dimensão ativo financeiro")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Define Delta Table Paths

In [10]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"

## Read Source Data

In [11]:
# Lê a tabela staging de cotação histórica
df_stg_cotacao_historica = spark.read.format("delta").load(delta_path_cotacao_historica)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cotacao_historica}")
print(f"Total de registros: {df_stg_cotacao_historica.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 14694933


## Transform Data

In [12]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(F.col("tp_mercado") == "010")

# Define a window function para particionar e ordenar
window_spec = Window.partitionBy(
    "cd_negociacao",
    "tp_mercado"
).orderBy(F.col("dt_pregao").desc())  # Ordena pela data mais recente primeiro

# Adiciona row_number para identificar o registro mais recente
df_with_row_number = df_stg_cotacao_historica.withColumn(
    "rn",
    F.row_number().over(window_spec)
)

# Filtra apenas o registro mais recente (row_number = 1)
df_most_recent = df_with_row_number.filter(F.col("rn") == 1)

# Gera chave surrogate e transforma os dados
df_dim_ativo = df_most_recent.select(
    # Gera surrogate key usando sha2 do cd_negociacao concatenado com nm_empresa
    F.sha2(F.concat(F.col("cd_negociacao"), F.col("nm_empresa")), 256).alias("sk_ativo"),
    F.col("cd_negociacao").alias("cd_ativo"),
    F.col("nm_empresa"),
    F.col("ds_especificacao_papel").alias("ds_ativo"),
    F.col("tp_mercado").alias("cd_tipo_mercado"),
    F.when(F.col("tp_mercado") == "010", "VISTA")
    .when(F.col("tp_mercado") == "012", "EXERCÍCIO DE OPÇÕES DE COMPRA")
    .when(F.col("tp_mercado") == "013", "EXERCÍCIO DE OPÇÕES DE VENDA")
    .when(F.col("tp_mercado") == "017", "LEILÃO")
    .when(F.col("tp_mercado") == "020", "FRACIONÁRIO")
    .when(F.col("tp_mercado") == "030", "TERMO")
    .when(F.col("tp_mercado") == "050", "FUTURO COM RETENÇÃO DE GANHO")
    .when(F.col("tp_mercado") == "060", "FUTURO COM MOVIMENTAÇÃO CONTÍNUA")
    .when(F.col("tp_mercado") == "070", "OPÇÕES DE COMPRA")
    .when(F.col("tp_mercado") == "080", "OPÇÕES DE VENDA")
    .otherwise("DESCONHECIDO")
    .alias("ds_tipo_mercado"),
    F.lit(None).cast("string").alias("cd_isin"),
    F.lit("AÇÃO").alias("ds_tipo_ativo")
)

## Write Data

In [13]:
# Carrega a tabela Delta existente
deltaTable = DeltaTable.forPath(spark, delta_path_dim_ativo)

# Realiza o merge (upsert) na tabela Delta
(
    deltaTable.alias("target")
    .merge(
        df_dim_ativo.alias("source"),
        "target.sk_ativo = source.sk_ativo"
    )
    # Quando o registro já existe e há diferença nos campos relevantes → atualiza
    .whenMatchedUpdate(
        condition="""
            coalesce(target.ds_ativo, '') != coalesce(source.ds_ativo, '') OR 
            coalesce(target.cd_tipo_mercado, '') != coalesce(source.cd_tipo_mercado, '') OR 
            coalesce(target.ds_tipo_mercado, '') != coalesce(source.ds_tipo_mercado, '') OR 
            coalesce(target.cd_isin, '') != coalesce(source.cd_isin, '') OR
            coalesce(target.ds_tipo_ativo, '') != coalesce(source.ds_tipo_ativo, '')
        """,
        set={
            "ds_ativo": "source.ds_ativo",
            "cd_tipo_mercado": "source.cd_tipo_mercado",
            "ds_tipo_mercado": "source.ds_tipo_mercado",
            "cd_isin": "source.cd_isin",
            "ds_tipo_ativo": "source.ds_tipo_ativo",
            "ts_atualizacao": "current_timestamp()"  # Atualiza timestamp no update
        }
    )
    # Quando o registro não existe → insere novo
    .whenNotMatchedInsert(values={
        "sk_ativo": "source.sk_ativo",
        "cd_ativo": "source.cd_ativo",
        "nm_empresa": "source.nm_empresa",
        "ds_ativo": "source.ds_ativo",
        "cd_tipo_mercado": "source.cd_tipo_mercado",
        "ds_tipo_mercado": "source.ds_tipo_mercado",
        "cd_isin": "source.cd_isin",
        "ds_tipo_ativo": "source.ds_tipo_ativo",
        "ts_insercao": "current_timestamp()",  # Data/hora da inserção
        "ts_atualizacao": "null"               # Ainda não foi atualizado
    })
    .execute()
)

# 📊 Obtém métricas do último merge
last_operation = deltaTable.history(1).select("operationMetrics").collect()[0][0]
inserted = int(last_operation.get("numTargetRowsInserted", 0))
updated = int(last_operation.get("numTargetRowsUpdated", 0))

print(f"✅ Merge realizado com sucesso")
print(f"🆕 Inseridos: {inserted}")
print(f"♻️ Atualizados: {updated}")

✅ Merge realizado com sucesso
🆕 Inseridos: 3200
♻️ Atualizados: 0


## Stop Spark Session

In [14]:
# Encerra a SparkSession
spark.stop()